In [1]:
import pymysql
from sqlalchemy import create_engine, text
from pandas import read_sql_query
from dateutil.relativedelta import relativedelta


from pydantic import (
    BaseSettings,
    Field
)

class Settings(BaseSettings):
    project_title: str = "Caris DATA API SERVICE"
    project_description: str = "REST API service for the datim reports and other relevant data analysis reports"
    project_version: str = "v0.0.1"
    project_docs_url: str = "/"
    
    DATIM_DESCRIPTION: str = "Get the datim data for dreams indicator - AGYW_PREV"
    DATIM_SUMMARY: str = "Get the datim data for dreams indicator - AGYW_PREV"

    DISTRICT_DESCRIPTION: str = "Get the district data for dreams indicator - AGYW_PREV"
    DISTRICT_SUMMARY: str = "Get the district data for dreams indicator - AGYW_PREV"
    
    
    mysql_username: str = Field(...,env='MYSQL_USERNAME')
    mysql_password: str = Field(...,env='MYSQL_PASSWORD')
    mysql_host: str = Field(...,env='MYSQL_HOST')
    mysql_port: int = Field(...,env='MYSQL_PORT')
    mysql_database: str = Field(...,env='MYSQL_DATABASE')
    
    commcare_username: str = Field(...,env='COMMCARE_USERNAME')
    commcare_password: str = Field(...,env='COMMCARE_PASSWORD')
    
    class Config:
        env_prefix = ""
        case_sensitive = False
        env_file = ".env"
        env_file_encoding = "utf-8"
        
        
        
settings = Settings()

In [2]:
EID = '''
select Patient_code,Code_mere,if(Code_mere in (select patient_code from patient),'yes','no') as Liaison_mere,left(patient_code,8) as Site_or_lab_code, hospital,Office, if(Network is not null,Network,'Autres') as Network, date_of_birth, gender, start_date as arv_start, if(Office = 'CAP', 'HUJ', lab) as lab, date_blood_taken, date_caris_received_sample, date_lab_received_sample, Result,
if(timestampdiff(day,date_of_birth,date_blood_taken)<=61,if(timestampdiff(day,date_of_birth,date_blood_taken)>=0,'0_2',''),if(timestampdiff(day,date_of_birth,date_blood_taken)<=365,if(timestampdiff(day,date_of_birth,date_blood_taken)>61,'2_12',''),'')) as tranche_age
,if(arv is not null,'yes','') as on_arv, timestampdiff(day,date_lab_received_sample,now()) as total_jours_Labo, timestampdiff(day,date_blood_taken,pcr_result_date) as from_specimen_to_result, timestampdiff(day,date_blood_taken,date_caris_received_sample) as from_specimen_to_caris,
timestampdiff(day,date_caris_received_sample,date_lab_received_sample) as from_caris_to_lab, timestampdiff(day,date_lab_received_sample,pcr_result_date) as from_lab_to_result,

if(!(pcr_result_date is null or pcr_result_date ='0000-00-00') and !(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00'),timestampdiff(day,date_lab_received_sample,pcr_result_date) , IF(!(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00'), timestampdiff(day,date_lab_received_sample,now()),"not_in_lab")) as days_in_lab,
case 
	when if(!(pcr_result_date is null or pcr_result_date ='0000-00-00') and !(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00'),timestampdiff(day,date_lab_received_sample,pcr_result_date) , IF(!(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00'), timestampdiff(day,date_lab_received_sample,now()),"not_in_lab"))>15 then 'greater than 15'
    when if(!(pcr_result_date is null or pcr_result_date ='0000-00-00') and !(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00'),timestampdiff(day,date_lab_received_sample,pcr_result_date) , IF(!(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00'), timestampdiff(day,date_lab_received_sample,now()),"not_in_lab"))<=15 then 'less or equal to 15'
    else'not in lab' end as Nbres_jours_Labo
   , 
   #timestampdiff(day,date_caris_received_sample,now()) as Total_jours_Caris,
   
if(!(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00'),timestampdiff(day,date_caris_received_sample,date_lab_received_sample),timestampdiff(day,date_caris_received_sample,now())) as days_between_caris_lab,
case 
	when if(!(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00') ,timestampdiff(day,date_caris_received_sample,date_lab_received_sample),timestampdiff(day,date_caris_received_sample,now())) >5 then 'greater than 5'
    when if(!(date_lab_received_sample is null or date_lab_received_sample ='0000-00-00'),timestampdiff(day,date_caris_received_sample,date_lab_received_sample),timestampdiff(day,date_caris_received_sample,now())) <=5 then 'less or equal to 5'
 end as Nbres_jours_Caris,

if(pcr_result=1,if(patient_info is not null,if(is_abandoned=0 and is_dead=0,'no','yes'),''),'') as pos_dead_abandon
,death_date
,abandoned_date
,pcr_result_date,
case 
    when month(date_blood_taken) between 10 and 12 then 'Q1'
    when month(date_blood_taken) between 1 and 3 then 'Q2'
    when month(date_blood_taken) between 4 and 6 then 'Q3'
    else 'Q4'
 End as Quarter,  -- month(date_blood_taken)  as month_blood_taken,
 #year(date_blood_taken) as year_test,
if( month(date_blood_taken) between 10 and 12 ,year(date_blood_taken)+1,year(date_blood_taken) ) as Year
from(
select patient_code,tme.id_patient as id_in_mere ,concat(tme.mother_city_code,'/',tme.mother_hospital_code,'/',tme.mother_code) as Code_mere, date_of_birth, tracking_infant.id_patient as patient_info, date_blood_taken, pcr_result,arv.id_patient as arv, is_dead, is_abandoned, death_date, abandoned_date, pcr_result_date, date_caris_received_sample, date_lab_received_sample,
lookup_testing_specimen_result.name as _result, if(lookup_testing_specimen_result.name="", 'En cours',lookup_testing_specimen_result.name) as Result,lookup_hospital.name as hospital,lookup_network.name as Network,lookup_hospital.office as Office, lookup_testing_gender.name as gender, tracking_regime.start_date, lookup_lab.name as lab from(
    select * from testing_specimen t1 where timestampdiff(day,date_of_birth,date_blood_taken)<=365 and date_blood_taken between '2014-10-01' and '2030-12-31'
    and date_blood_taken=(select min(date_blood_taken) from testing_specimen t2 where t1.id_patient=t2.id_patient)
)test_1
left join patient pa on pa.id=test_1.id_patient
left join testing_mereenfant tme on pa.id= tme.id_patient
left join
(select * from(
    select id_patient from tracking_followup where on_arv=1
    union
    select id_patient from tracking_regime where category='regime_infant_treatment'
    )o
)arv on arv.id_patient=test_1.id_patient
left join tracking_infant on tracking_infant.id_patient=test_1.id_patient
left join lookup_testing_specimen_result on pcr_result=lookup_testing_specimen_result.id
left join lookup_hospital on concat(lookup_hospital.city_code,'/',lookup_hospital.hospital_code)=left(patient_code,8)
left join tracking_regime on tracking_regime.id_patient=test_1.id_patient
left join lookup_testing_gender on lookup_testing_gender.id=gender
left join lookup_lab on lookup_hospital.id_lab=lookup_lab.id
left join lookup_network on lookup_hospital.network=lookup_network.id
where timestampdiff(day,test_1.date_of_birth,test_1.date_blood_taken)<=365 and test_1.date_blood_taken between '2014-10-01' and '2030-12-31' and linked_to_id_patient=0
group by test_1.id_patient) final
'''

engine = create_engine(
    f"mysql+pymysql://{settings.mysql_username}:{settings.mysql_password}@{settings.mysql_host}/{settings.mysql_database}")


eid = read_sql_query(text(EID), engine.connect(), parse_dates=True)

# close the pool of connection
engine.dispose()

In [3]:
eid.columns

Index(['Patient_code', 'Code_mere', 'Liaison_mere', 'Site_or_lab_code',
       'hospital', 'Office', 'Network', 'date_of_birth', 'gender', 'arv_start',
       'lab', 'date_blood_taken', 'date_caris_received_sample',
       'date_lab_received_sample', 'Result', 'tranche_age', 'on_arv',
       'total_jours_Labo', 'from_specimen_to_result', 'from_specimen_to_caris',
       'from_caris_to_lab', 'from_lab_to_result', 'days_in_lab',
       'Nbres_jours_Labo', 'days_between_caris_lab', 'Nbres_jours_Caris',
       'pos_dead_abandon', 'death_date', 'abandoned_date', 'pcr_result_date',
       'Quarter', 'Year'],
      dtype='object')

In [4]:
eid.rename(columns = {'Site_or_lab_code' : 'CODE LABO'}, inplace = True)


In [5]:
#eid.tranche_age.unique()
eid.Result.unique()

array(['ADN VIH-1 Non-Detècté', 'En cours', 'ADN VIH-1 Detècté',
       'Autre DBS'], dtype=object)

In [6]:
eid['calculated_ageDiff'] = eid.apply(lambda eid: relativedelta(eid.date_blood_taken, eid.date_of_birth).months,axis=1)
eid['calculated_ageDiff'] = eid['calculated_ageDiff'].abs()

In [7]:
eid.calculated_ageDiff.unique()

array([ 1,  6, 10,  0,  8,  7,  5,  9,  2, 11,  4,  3])

In [8]:
eid.loc[(eid.calculated_ageDiff>=0) & (eid.calculated_ageDiff<=2),"tranche_age"] = "0_2"
eid.loc[(eid.calculated_ageDiff>2) & (eid.calculated_ageDiff<=12),"tranche_age"] = "2_12"

In [9]:
eid.tranche_age.unique()

array(['0_2', '2_12'], dtype=object)

In [10]:
#----------------------------------------------------------------------------------------------------------------------

period_1 = 'Q1'
year = 2023
start_date = '1er Octobre'



In [16]:
eid.Year.unique().tolist()
#eid.Quarter.unique().tolist()
#eid.Network.unique().tolist()

[2016, 2015, 2017, 2018, 2021, 2019, 2022, 2020, 2023]

In [61]:
def pmtctEID_heiPos_ON_ARV(eid, year=None):
    if year == None:
        eid_Qi = eid
    else:
        eid_Qi = eid[(eid['Year'] == year)]


    #Nombre de spécimens par tranche d'âge
    PT = eid_Qi.pivot_table(values='Patient_code', index = 'tranche_age', aggfunc = len, fill_value = 0, margins = True, margins_name ='Total')
    PT.rename(columns = {'Patient_code' : 'Spécimens prélevés'}, inplace = True)

    #Pourcentage
    PTp = round(PT.div(PT.iloc[-1,:], axis = 1), 2)*100
    PTp.rename(columns = {'Spécimens prélevés' : '% Spécimens prélevés'}, inplace = True)

    # Nombre de positif
    eid_positive = eid_Qi[eid_Qi['Result'] == 'ADN VIH-1 Detècté']
    PT2 = eid_positive.pivot_table(values = 'Patient_code', index = 'tranche_age', aggfunc = len, margins = True, margins_name ='Total')
    PT2.rename(columns = {'Patient_code' : 'Enfants positifs'}, inplace = True)

    #Nombre sous ARV
    eid_on_arv = eid_positive[eid_positive['on_arv'] == 'yes']
    PT3 = eid_on_arv.pivot_table(values = 'Patient_code', index = 'tranche_age', aggfunc = len, margins = True, margins_name = 'Total')
    PT3.rename(columns = {'Patient_code' : 'Sous ARV'}, inplace = True)

    #Enfants sans résultat
    eid_no_result = eid_Qi[(eid_Qi['Result'] != 'ADN VIH-1 Non-Detècté') & (eid_Qi['Result'] != 'ADN VIH-1 Detècté')]
    PT4 = eid_no_result.pivot_table(values = 'Patient_code', index = 'tranche_age', aggfunc = len, margins = True, margins_name = 'Total')
    PT4.rename(columns = {'Patient_code' : 'Sans résultat'}, inplace =True)
    
    PT4p = round(PT4.div(PT4.iloc[-1,:], axis = 1), 2)*100
    PT4p.rename(columns = {'Sans résultat' : '% Sans résultat'}, inplace = True)

    #Concaténation des données et tableau final
    #EID = PT.merge(PTp, on = 'tranche_age', how = 'left').merge(PT2, on = 'tranche_age', how = 'left').merge(PT3, on = 'tranche_age', how = 'left').merge(PT4, on = 'tranche_age', how = 'left')
    EID = PT.merge(PTp, on = 'tranche_age', how = 'left').merge(PT2, on = 'tranche_age', how = 'left').merge(PT3, on = 'tranche_age', how = 'left').merge(PT4, on = 'tranche_age', how = 'left').merge(PT4p, on = 'tranche_age', how = 'left')
    EID.reset_index(inplace = True)
    EID = EID.fillna(0)
    return EID



#pmtctEID_heiPos_ON_ARV(eid,2023)
pmtctEID_heiPos_ON_ARV(eid)

,tranche_age,Spécimens prélevés,% Spécimens prélevés,Enfants positifs,Sous ARV,Sans résultat,% Sans résultat
0,0_2,20873,79.0,658,537,854,78.0
1,2_12,5711,21.0,647,453,243,22.0
2,Total,26584,100.0,1305,990,1097,100.0


In [13]:
table = pmtctEID_heiPos_ON_ARV(eid)

table.to_dict('records')


[{'tranche_age': '0_2',
  'Spécimens prélevés': 20873,
  '% Spécimens prélevés': 79.0,
  'Enfants positifs': 658,
  'Sous ARV': 537,
  'Sans résultat': 854,
  '% Sans résultat': 78.0},
 {'tranche_age': '2_12',
  'Spécimens prélevés': 5711,
  '% Spécimens prélevés': 21.0,
  'Enfants positifs': 647,
  'Sous ARV': 453,
  'Sans résultat': 243,
  '% Sans résultat': 22.0},
 {'tranche_age': 'Total',
  'Spécimens prélevés': 26584,
  '% Spécimens prélevés': 100.0,
  'Enfants positifs': 1305,
  'Sous ARV': 990,
  'Sans résultat': 1097,
  '% Sans résultat': 100.0}]

In [74]:

class Eid():
    
    @classmethod
    def get_filter_title(cls,eid):
        year = eid.Year.unique().tolist()
        quarter = eid.Quarter.unique().tolist()
        network = eid.Network.unique().tolist()
        return {
            "year_title": year,
            "quarter_title": quarter,
            "network_title": network
        }
    
    @classmethod
    def pmtctEID_heiPos_ON_ARV(cls,eid, year=None,quarter=None, network=None):
        if (year is None and quarter is not None and network is not None):
            eid_Qi = eid[(eid.Quarter==quarter)&(eid.Network==network)]
        elif(quarter is None and year is not None and network is not None):
            eid_Qi = eid[(eid.Year==year)&(eid.Network==network)]
        elif(network is None and year is not None and quarter is not None):
            eid_Qi = eid[(eid.Year==year)&(eid.Quarter==quarter)]
        elif(year is not None and quarter is None and network is None):
            eid_Qi = eid[(eid.Year==year)]
        elif(quarter is not None and year is None and network is None):
            eid_Qi = eid[(eid.Quarter==quarter)]
        elif(network is not None and year is None and quarter is None):
            eid_Qi = eid[(eid.Network==network)]
        elif(year is not None and quarter is not None and network is not None):
            eid_Qi = eid[(eid.Year==year)&(eid.Quarter==quarter)&(eid.Network==network)]
        else:
            eid_Qi = eid
        
        #Nombre de spécimens par tranche d'âge
        PT = eid_Qi.pivot_table(values='Patient_code', index = 'tranche_age', aggfunc = len, fill_value = 0, margins = True, margins_name ='Total')
        PT.rename(columns = {'Patient_code' : 'Spécimens prélevés'}, inplace = True)

        #Pourcentage
        PTp = round(PT.div(PT.iloc[-1,:], axis = 1), 2)*100
        PTp.rename(columns = {'Spécimens prélevés' : '% Spécimens prélevés'}, inplace = True)

        # Nombre de positif
        eid_positive = eid_Qi[eid_Qi['Result'] == 'ADN VIH-1 Detècté']
        PT2 = eid_positive.pivot_table(values = 'Patient_code', index = 'tranche_age', aggfunc = len, margins = True, margins_name ='Total')
        PT2.rename(columns = {'Patient_code' : 'Enfants positifs'}, inplace = True)

        #Nombre sous ARV
        eid_on_arv = eid_positive[eid_positive['on_arv'] == 'yes']
        PT3 = eid_on_arv.pivot_table(values = 'Patient_code', index = 'tranche_age', aggfunc = len, margins = True, margins_name = 'Total')
        PT3.rename(columns = {'Patient_code' : 'Sous ARV'}, inplace = True)

        #Enfants sans résultat
        eid_no_result = eid_Qi[(eid_Qi['Result'] != 'ADN VIH-1 Non-Detècté') & (eid_Qi['Result'] != 'ADN VIH-1 Detècté')]
        PT4 = eid_no_result.pivot_table(values = 'Patient_code', index = 'tranche_age', aggfunc = len, margins = True, margins_name = 'Total')
        PT4.rename(columns = {'Patient_code' : 'Sans résultat'}, inplace =True)
        
        PT4p = round(PT4.div(PT4.iloc[-1,:], axis = 1), 2)*100
        PT4p.rename(columns = {'Sans résultat' : '% Sans résultat'}, inplace = True)

        #Concaténation des données et tableau final
        #EID = PT.merge(PTp, on = 'tranche_age', how = 'left').merge(PT2, on = 'tranche_age', how = 'left').merge(PT3, on = 'tranche_age', how = 'left').merge(PT4, on = 'tranche_age', how = 'left')
        EID = PT.merge(PTp, on = 'tranche_age', how = 'left').merge(PT2, on = 'tranche_age', how = 'left').merge(PT3, on = 'tranche_age', how = 'left').merge(PT4, on = 'tranche_age', how = 'left').merge(PT4p, on = 'tranche_age', how = 'left')
        EID.reset_index(inplace = True)
        EID = EID.fillna(0)
        return EID 



In [91]:
Eid.pmtctEID_heiPos_ON_ARV(eid,year=2023,quarter='Q2', network= 'CMMB')

,tranche_age,Spécimens prélevés,% Spécimens prélevés,Enfants positifs,Sous ARV,Sans résultat,% Sans résultat
0,0_2,60,90.0,0.0,0.0,44,92.0
1,2_12,7,10.0,0.0,0.0,4,8.0
2,Total,67,100.0,0.0,0.0,48,100.0


In [83]:
Eid.get_filter_title(eid)["network_title"]


['Santé',
 'CMMB',
 'UGP',
 'PIH',
 'GHESKIO',
 'Autres',
 'Fondation Saint-Luc',
 'MSPP',
 'CDS',
 'FOSREF',
 'I-TECH',
 'ICAP']

In [84]:
Eid.get_filter_title(eid)["quarter_title"]


['Q1', 'Q3', 'Q4', 'Q2']

In [85]:
Eid.get_filter_title(eid)["year_title"]

[2016, 2015, 2017, 2018, 2021, 2019, 2022, 2020, 2023]